In [28]:
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from chat import init_model as init_nanoGPT
from chat import respond as get_respond_nanoGPT
import torch
from bert_score import score

In [29]:
data_path = '../../../data/emotion/validation/100_validation.csv'
df = pd.read_csv(data_path)

In [30]:
df.head()

,Situation,grouped_emotion,empathetic_dialogues,labels
0,Last night I heard strange noises coming from ...,afraid,In the middle of the night I heard some weird ...,Should have grabbed the gun.
1,My mom and sister threw me a baby shower when ...,excited,that was very nice of them congratulations,"Thank you! It was so nice, I had no idea it w..."
2,I just applied for a new job. After the inter...,grateful,"Oh really, do you feel like you did a great job?",I do! I'm feeling very optimistic about it
3,I loaned some money to my friend at work. Turn...,annoyed,Wow! What a jerk for him to up and leave with ...,"It was a medium amount of money but still, he ..."
4,I was out walking late last night and seen som...,NaN,Oh my god. What happened?,"Well, I started walking much faster. It looked..."


## Get model response

In [31]:
model_list = {
    'withoutemotion_single': 'withoutemotion/singleConversation',
    'withoutemotion_whole':'withoutemotion/wholeConversation',
    'withemotion':'withemotion',
    'withcontext': 'withcontext'
}
for model_type, model_path in model_list.items():
    model_list[model_type] = init_nanoGPT(model_path)


Loading model from: ../trained-saved/withoutemotion/singleConversation/ckpt.pt
number of parameters: 3.42M
Loading model from: ../trained-saved/withoutemotion/wholeConversation/ckpt.pt


/Users/yanxi.lin/Downloads/Project-ML/src/models/nanoGPT/chat.py:40: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(ckpt_path, map_location=device)


number of parameters: 3.42M
Loading model from: ../trained-saved/withemotion/ckpt.pt
number of parameters: 3.42M
Loading model from: ../trained-saved/withcontext/ckpt.pt
number of parameters: 3.42M


In [32]:
def get_response_from_nanoGPT(row,model):
    situation = row['Situation']
    emotion = row['grouped_emotion']
    human = row['empathetic_dialogues']
    start = '<bot> ' + human + '<human>'
    response, new_emotion, new_context = get_respond_nanoGPT(start, 1, model=model, enable_print=False)
    return response #, new_emotion, new_context

In [33]:
for model_type, model in model_list.items():
    label = 'new_label_' + model_type
    df[label] = df.apply(lambda row: get_response_from_nanoGPT(row, model), axis=1)

In [ ]:
# df.to_csv('./evaluation_result.csv', index=False)

In [34]:
df.head()

,Situation,grouped_emotion,empathetic_dialogues,labels,new_label_withoutemotion_single,new_label_withoutemotion_whole,new_label_withemotion,new_label_withcontext
0,Last night I heard strange noises coming from ...,afraid,In the middle of the night I heard some weird ...,Should have grabbed the gun.,Oh no! Was it a nasty thing?,,I've been there and I think it's a bit scary.,I would have been a bad one.
1,My mom and sister threw me a baby shower when ...,excited,that was very nice of them congratulations,"Thank you! It was so nice, I had no idea it w...",It was very nice,Thank you. I'm happy with that.,Yeah they really did. I was so excited and ha...,"Yes, it was a big surprise. I was so happy fo..."
2,I just applied for a new job. After the inter...,grateful,"Oh really, do you feel like you did a great job?",I do! I'm feeling very optimistic about it,"I do, I hope I get it.",I am very prepared. I will be ready for the r...,I did. It was so nice. It was so nice that I ...,"I did, and I was really happy to find it."
3,I loaned some money to my friend at work. Turn...,annoyed,Wow! What a jerk for him to up and leave with ...,"It was a medium amount of money but still, he ...",It was a good idea!,Yes! He was very lucky.,"It was a promotion, but I was not sure it was...",I was so mad. He was a jerk.
4,I was out walking late last night and seen som...,NaN,Oh my god. What happened?,"Well, I started walking much faster. It looked...","I was sitting there, but I'm not sure if it w...",It was so gross. It was so gross!,I was in a car accident. I was so upset.,"Oh my gosh, I hope they don't have to help me."


## BLEU

In [35]:
def get_bleu(compared_column):
    bleu_scores = []
    smoothing_function = SmoothingFunction().method1  # To avoid 0 scores due to short sentences
    for _, row in df.iterrows():
        for ref, output in zip(row['labels'], row[compared_column]):
            # Tokenize each sentence (split by words)
            reference_tokens = [ref.split()]  # BLEU expects a list of lists for references
            output_tokens = output.split()
            
            # Calculate BLEU score
            bleu = sentence_bleu(reference_tokens, output_tokens, smoothing_function=smoothing_function)
            bleu_scores.append(bleu)
    return bleu_scores

In [ ]:
bleu_scores = {}
bleu_scores_average = {}
for model_type, model in model_list.items():
    label = 'new_label_' + model_type
    # print(label)
    bleu_scores[model_type] = get_bleu(label)
    bleu_scores_average[model_type] = sum(bleu_scores[model_type]) / len(bleu_scores[model_type])

new_label_withoutemotion_single
new_label_withoutemotion_whole
new_label_withemotion
new_label_withcontext


In [ ]:
bleu_scores_average

{'withoutemotion_single': 0.006172121992169218,
 'withoutemotion_whole': 0.005432207996084309,
 'withemotion': 0.005625406761168866,
 'withcontext': 0.0061107630233090135}

## BertScore

In [ ]:
def calculate_bert_score(compared_column):
    # Check for MPS device
    device = "mps" if torch.backends.mps.is_available() else "cpu"
    print(f"Using device: {device}")

    model_outputs = df['labels']
    reference_sentences = df[compared_column]

    if len(model_outputs) != len(reference_sentences):
        raise ValueError("Mismatch in lengths: model_outputs and reference_sentences must be of the same length.")
    # Convert model outputs and reference sentences to strings
    model_outputs = [str(output) for output in model_outputs]
    reference_sentences = [str(ref) for ref in reference_sentences]
    # Calculate precision, recall, and F1 for each pair of reference and output
    P, R, F1 = score(model_outputs, reference_sentences, lang='en', verbose=True, device = device)
    return P, R, F1

In [50]:
bert_scores = {}

for model_type, model in model_list.items():
    label = 'new_label_' + model_type
    bert_scores[model_type] = {}
    
    # Calculate BERT score and assign it to the dictionary
    bert_scores[model_type]['P'], bert_scores[model_type]['R'], bert_scores[model_type]['F1'] = calculate_bert_score(label)

Using device: mps


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/4 [00:00<?, ?it/s]

computing greedy matching.


...

calculating scores...
computing bert embedding.


  0%|          | 0/4 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/2 [00:00<?, ?it/s]

done in 1.18 seconds, 84.41 sentences/sec


In [53]:
for model_type, model in model_list.items():
    label = 'new_label_' + model_type
    # P avarage
    P_average = sum(bert_scores[model_type]['P']) / len(bert_scores[model_type]['P'])
    # R avarage
    R_average = sum(bert_scores[model_type]['R']) / len(bert_scores[model_type]['R'])
    # F1 avarage
    F1_average = sum(bert_scores[model_type]['F1']) / len(bert_scores[model_type]['F1'])
    print("--------------------------------------------------")
    print(f"Model: {model_type}")
    print(f"Average Precision: {P_average}")
    print(f"Average Recall: {R_average}")
    print(f"Average F1: {F1_average}")
    

--------------------------------------------------
Model: withoutemotion_single
Average Precision: 0.8552975654602051
Average Recall: 0.8628495931625366
Average F1: 0.8588349223136902
--------------------------------------------------
Model: withoutemotion_whole
Average Precision: 0.7506046891212463
Average Recall: 0.7574512958526611
Average F1: 0.7538419961929321
--------------------------------------------------
Model: withemotion
Average Precision: 0.8577821254730225
Average Recall: 0.8660477995872498
Average F1: 0.861606776714325
--------------------------------------------------
Model: withcontext
Average Precision: 0.8571752309799194
Average Recall: 0.8627082705497742
Average F1: 0.8597039580345154


## # GLUE - Sentiment Analysis Evaluation (SST-2)

In [55]:
from transformers import pipeline
def evaluate_sentiment(compared_column):
    device = "mps" if torch.backends.mps.is_available() else "cpu"
    print(f"Using device: {device}")
    
    # Load multi-class sentiment or emotion pipeline
    sentiment_pipeline = pipeline(
        "text-classification", 
        model="bhadresh-savani/distilbert-base-uncased-emotion", 
        device=0 if device == "mps" else -1
    )
    
    scores = []
    model_outputs = df['labels']
    reference_sentences = df[compared_column]

    if len(model_outputs) != len(reference_sentences):
        raise ValueError("Mismatch in lengths: model_outputs and reference_sentences must be of the same length.")
    # Convert model outputs and reference sentences to strings
    model_outputs = [str(output) for output in model_outputs]
    reference_sentences = [str(ref) for ref in reference_sentences]

    for i, (output, reference) in enumerate(zip(model_outputs, reference_sentences), start=1):
        output_sentiment = sentiment_pipeline(output)[0]['label']
        reference_sentiment = sentiment_pipeline(reference)[0]['label']
        
        score = 1 if output_sentiment == reference_sentiment else 0
        scores.append(score)

    return scores


In [56]:
sentiment_scores = {}

for model_type, model in model_list.items():
    label = 'new_label_' + model_type
    sentiment_scores[model_type] = {}
    
    sentiment_scores[model_type] = evaluate_sentiment(label)

Using device: mps
Using device: mps
Using device: mps
Using device: mps


In [58]:
for model_type, model in model_list.items():
    label = 'new_label_' + model_type
    GLUE_average = sum(sentiment_scores[model_type]) / len(sentiment_scores[model_type])
    print("--------------------------------------------------")
    print(f"Model: {model_type}")
    print(f"Average Sentiment Score: {GLUE_average}")
    

--------------------------------------------------
Model: withoutemotion_single
Average Sentiment Score: 0.52
--------------------------------------------------
Model: withoutemotion_whole
Average Sentiment Score: 0.49
--------------------------------------------------
Model: withemotion
Average Sentiment Score: 0.55
--------------------------------------------------
Model: withcontext
Average Sentiment Score: 0.45
